<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Power-by-'Time-Use-Survey'-(ZVE,-only-2015)" data-toc-modified-id="Power-by-'Time-Use-Survey'-(ZVE,-only-2015)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Power by 'Time Use Survey' (ZVE, only 2015)</a></span></li><li><span><a href="#Power-by-'Standard-Load-Profiles'-(SLP,-all-years)" data-toc-modified-id="Power-by-'Standard-Load-Profiles'-(SLP,-all-years)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Power by 'Standard Load Profiles' (SLP, all years)</a></span></li><li><span><a href="#Gas-by-'Standard-Load-Profiles'-(SLP,-all-years)" data-toc-modified-id="Gas-by-'Standard-Load-Profiles'-(SLP,-all-years)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Gas by 'Standard Load Profiles' (SLP, all years)</a></span></li><li><span><a href="#Older-version" data-toc-modified-id="Older-version-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Older version</a></span></li></ul></div>

In [ ]:
#Import packages
from disaggregator import config, data, spatial, temporal, plot
import pandas as pd
import matplotlib.pyplot as plt

# Power by 'Time Use Survey' (ZVE, only 2015)

Get spatial data in [MWh] that shall be disaggregated using `disaggregator.temporal`.

In [ ]:
df_spat = spatial.disagg_households_power('population', weight_by_income=True) * 1e3
df_spat.head()

Calculate temporal distribution keys based on the ZVE. They will be saved in your `data_out` folder. This step may take up to 60 Minutes but is only required once.

In [ ]:
temporal.make_zve_load_profiles()

Then load the temporal distribution keys based on the ZVE:

In [ ]:
df_temp = (pd.read_csv(config.data_out('ZVE_timeseries_AllRegions_2015.csv'),
                       engine='c', index_col=0, parse_dates=True,
                       infer_datetime_format=True)
              .pipe(data.transpose_spatiotemporal))
df_temp

The function `temporal.disagg_temporal()` needs two inputs:
1. **a spatial dataset**  
    either as `pd.Series` or `pd.DataFrame`: in both cases _NUTS-3-index'ed_


2. **a temporal dataset**  
    if passed as `pd.DataFrame`: _NUTS-3-index'ed_  
    if passed as `pd.Series`: _pd.DateTimeIndex'ed_

Pre-check:
* Both `df_spat` and `df_temp` are `pd.DataFrame`s
* Both have NUTS-3-indices.

So now we can directly perfom the disaggregation:

In [ ]:
df_elc_tempo_ZVE = temporal.disagg_temporal(df_spat, df_temp)

In [ ]:
df_elc_tempo_ZVE.head()

We see that the data has successfully been disaggregated. However, regions are in rows and time-steps are in columns -- this is not really intuitive. Better would be the other way round, right? We can achieve this by passing `time_indexed=True` to give us a transponed version:

In [ ]:
df_elc_tempo_ZVE = temporal.disagg_temporal(df_spat, df_temp, time_indexed=True)

In [ ]:
df_elc_tempo_ZVE.head()

So let's take out one region, e.g. Berlin (DE300), and plot it as a line graph:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 4))
df_elc_tempo_ZVE['DE300'].plot(ax=ax)
plot.set_ax_format(ax=ax, xlim=(0, 8760), ylim=(0, None), 
                   ylabel='[MWh]', xlabel='Hour of Year',
                   axtitle='Power Consumption of Private Households in Berlin')

This depictation however does not provide a good visualization of what is happening on shorter timescales, i.e. within one day.

Therefore, we choose to plot it as a heatmap:

In [ ]:
fig, ax = plot.heatmap_timeseries(df_elc_tempo_ZVE['DE300'], clabel='[MWh]')

# Power by 'Standard Load Profiles' (SLP, all years)
(Updated Dec 2020)

Use `temporal.disagg_temporal_power_housholds_slp()` to get power demand (2016) of households in 15min slices in **MWh**.

In [ ]:
df_elec_temp_slp = temporal.disagg_temporal_power_housholds_slp(use_nuts3code=True, by='population', weight_by_income=False,  year=2016)
#df_elec_temp_slp.head()

In [ ]:
df_elec_temp_slp.head()

... and can then be plotted directly:

In [ ]:
fig, ax = plot.heatmap_timeseries(df_elec_temp_slp.resample('H').sum()['DE300'], clabel='[MWh]')
ax[0, 0].set_title('Electricity Load Private Households Germany')

# Gas by 'Standard Load Profiles' (SLP, all years)
(Updated Dec 2020)

Use `disagg_temporal_gas_households()` to get hourly gas demand of households in **MWh** for every region. Function uses spatial function `disagg_households_gas()`, which distinguishes between gas demand for heating, cooking and warm water. However, demands are summed up for SLP.

Demands are then split between demand for _single family houses (EFH)_ and _multi family houses (MFH)_ with the help of ``data.percentage_EFH_MFH()``, which calculates the percentages of EFH and MFH per NUTS-3 based on the available living spaces.

In [ ]:
# use function disagg_daily_gas_slp_households similar as for CTS to create gas SLP for all years
# specify year either through config file (restart kernel after) or specify directly in function
df_households_gas = temporal.disagg_temporal_gas_households(use_nuts3code=True, year=2017)

In [ ]:
df_households_gas.head()

You can also plot `df_households_gas` directly:

In [ ]:
fig, ax = plot.heatmap_timeseries(df_households_gas.sum(axis=1)/1e3, clabel='[GWh]')
ax[0, 0].set_title('Gas Load Private Households Germany')

# Older version

Normalized gas SLP for _single family houses_ (EFH) and _multi family houses (MFH)_ have been uploaded into the database untill 2016 as well. which could be used with ``temporal.disagg_temporal(df_spat, df_temp)`` and ``data.transpose_spatiotemporal()`` to perform the same disaggregation.

In [ ]:
df_temp_EFH = data.standard_load_profile_gas(typ='EFH') # type='EFH'
df_temp_EFH.head()

This dataframe has the NUTS-3-regions in columns and the timesteps in rows. Though being intuitive that way, the function ` temporal.disagg_temporal(df_spat, df_temp)` requires us to provide the temporal dataset _NUTS-3-index'ed_ (i.e. with timesteps in columns) for compatibility reasons. To achieve that, we use the function `data.transpose_spatiotemporal()`

In [ ]:
df_temp_MFH = data.standard_load_profile_gas(typ='MFH') # type='MFH'

In [ ]:
df_temp_MFH.head()

In [ ]:
df_temp_EFH = data.transpose_spatiotemporal(df_temp_EFH)
df_temp_MFH = data.transpose_spatiotemporal(df_temp_MFH)

In [ ]:
#define df_gas_EFH and df_gas_MFH
#df_gas_EFH =
#df_gas_MFH =

Note: It is by design that the pd.DateTimeIndex gets lost while transposing this way for a better handling of the data. The former index is being recovered, one it is transposed again.

In [ ]:
df_gas_st_EFH = temporal.disagg_temporal(df_gas_EFH, df_temp_EFH, time_indexed=True)
df_gas_st_MFH = temporal.disagg_temporal(df_gas_MFH, df_temp_MFH, time_indexed=True)
#df_gas_st_EFH.head()

Sum these consumption for both building types up:

In [ ]:
df_gas_st = df_gas_st_EFH + df_gas_st_MFH
df_gas_st

Instead of selecting one specific region, this time we choose to sum it up for the entire country:

In [ ]:
ser_gas = df_gas_st.sum(axis=1)

Now we can plot this one again as well:

In [ ]:
fig, ax = plot.heatmap_timeseries(ser_gas/1e3, clabel='[GWh]')
ax[0, 0].set_title('Gas Load Private Households Germany')